# Capstone Final Project Week 1

### Import Main Modules

In [1]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
import requests # library to handle requests

import matplotlib as mpl
import matplotlib.pyplot as plt
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import json # library to handle JSON files

import folium # plotting library

%matplotlib inline 

Neighborhood data is from  https://cohgis-mycity.opendata.arcgis.com/datasets/super-neighborhoods?selectedAttribute=RECOGNITIO
The shapefile data has been converted to geo json data.

In [2]:
with open('Super_Neighborhoods.json') as json_data:
    hou_geodata = json.load(json_data)

### Get the center coorindates of each neighborhood
The geo json gives the boundaries of neighborhood, but no center point coorindates. Input the json to a dataframe and calculate the center.
Mean of coorindates is the center of the area.

Pay attention of the layers of the json dictionary data structure

In [3]:
df=pd.DataFrame(hou_geodata['features'])
print(df.shape)
df.head()

(88, 4)


,type,geometry,properties,id
0,Feature,"{'type': 'Polygon', 'coordinates': [[[-95.3858...","{'OBJECTID': 1, 'PERIMETER': 16572.025, 'POLYI...",1
1,Feature,"{'type': 'Polygon', 'coordinates': [[[-95.3201...","{'OBJECTID': 2, 'PERIMETER': 43118.77, 'POLYID...",2
2,Feature,"{'type': 'Polygon', 'coordinates': [[[-95.3473...","{'OBJECTID': 3, 'PERIMETER': 39256.388, 'POLYI...",3
3,Feature,"{'type': 'Polygon', 'coordinates': [[[-95.2562...","{'OBJECTID': 4, 'PERIMETER': 59784.698, 'POLYI...",4
4,Feature,"{'type': 'Polygon', 'coordinates': [[[-95.4538...","{'OBJECTID': 5, 'PERIMETER': 75759.39, 'POLYID...",5


In [4]:
hou_df=df.rename(columns={"type":"SNBNAME","geometry":"Latitude","properties":"Longitude","id":"POLYID"})

Update the DF using the json data

In [5]:
i=0
for i in range(hou_df.shape[0]):
     hou_df.loc[i,"SNBNAME"]=hou_geodata['features'][i]['properties']["SNBNAME"]
     hou_df.loc[i,"Latitude"]=pd.DataFrame(hou_geodata['features'][i]['geometry']['coordinates'][0]).mean()[1] 
     hou_df.loc[i,"Longitude"]=pd.DataFrame(hou_geodata['features'][i]['geometry']['coordinates'][0]).mean()[0] 
     hou_df.loc[i,"POLYID"]=hou_geodata['features'][i]['properties']["POLYID"]

In [6]:
hou_df.head()

,SNBNAME,Latitude,Longitude,POLYID
0,FOURTH WARD,29.7591,-95.3802,60
1,SECOND WARD,29.7517,-95.3316,63
2,DOWNTOWN,29.7583,-95.3539,61
3,CLINTON PARK TRI-COMMUNITY,29.7398,-95.2559,59
4,GREATER UPTOWN,29.7713,-95.4901,21


In [7]:
hou_df.shape

(88, 4)

Get Houston coordinates to plot map

In [8]:
address = 'Houston, TX'

geolocator = Nominatim(user_agent="hou_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of H-Town are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of H-Town are 29.7589382, -95.3676974.


In [9]:
map_hou = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, polyid, neighborhood in zip(hou_df['Latitude'], hou_df['Longitude'], hou_df['POLYID'], hou_df['SNBNAME']):
    label = '{}, {}'.format(neighborhood, polyid)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hou)  
    
map_hou

Cluster Super neighborhoods on the map. When you zoom in/out a different level, neighborhoods will be clustered. 

In [10]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
hou_map_clst = folium.Map(location = [latitude, longitude], zoom_start = 9)

# instantiate a mark cluster object for the super neighborhood in the dataframe
SNBNAMEs = plugins.MarkerCluster().add_to(hou_map_clst)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(hou_df['Latitude'], hou_df['Longitude'], hou_df['SNBNAME']):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(SNBNAMEs)

# display map
hou_map_clst

Choroplith map to show the neighborhood boundaries. 

In [13]:
hou_map_chor = folium.Map(location = [latitude, longitude], zoom_start = 10)

In [14]:
hou_map_chor.choropleth(
    geo_data=hou_geodata,
    data=hou_df,
    columns=['SNBNAME', 'POLYID'],       # Color scale is the neighborhood POLYID, no other meanings
    key_on='feature.properties.SNBNAME',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Houston Super Neighborhood'
)

# display map
hou_map_chor